<a href="https://colab.research.google.com/github/sharan11/Intro-to-Deep-Learning-with-Pytorch-Exercise-Solutions/blob/master/Character_Level_RNN_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character-Level LSTM in PyTorch

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

Load in Data

In [0]:
# open text file and read in data as `text`
with open('anna.txt', 'r') as f:
    text = f.read()

In [3]:
text[:300]

"Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverything was in confusion in the Oblonskys' house. The wife had\ndiscovered that the husband was carrying on an intrigue with a French\ngirl, who had been a governess in their family, and she had announced to\n"

Tokenization

In [0]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [6]:
encoded[:100]

array([34, 54, 25, 18, 81, 74, 70, 23, 71, 58, 58, 58, 12, 25, 18, 18, 59,
       23, 72, 25,  1, 31, 48, 31, 74, 62, 23, 25, 70, 74, 23, 25, 48, 48,
       23, 25, 48, 31, 75, 74, 47, 23, 74, 26, 74, 70, 59, 23, 43, 44, 54,
       25, 18, 18, 59, 23, 72, 25,  1, 31, 48, 59, 23, 31, 62, 23, 43, 44,
       54, 25, 18, 18, 59, 23, 31, 44, 23, 31, 81, 62, 23, 19, 46, 44, 58,
       46, 25, 59, 73, 58, 58, 28, 26, 74, 70, 59, 81, 54, 31, 44])

Pre-processing the data

one-hot encoding

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [8]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


Creating Batches

**TODO**: Write the code for creating batches in the function below.

In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

Test your implementation

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [11]:

# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[34 54 25 18 81 74 70 23 71 58]
 [62 19 44 23 81 54 25 81 23 25]
 [74 44 61 23 19 70 23 25 23 72]
 [62 23 81 54 74 23 65 54 31 74]
 [23 62 25 46 23 54 74 70 23 81]
 [65 43 62 62 31 19 44 23 25 44]
 [23 78 44 44 25 23 54 25 61 23]
 [45 49 48 19 44 62 75 59 73 23]]

y
 [[54 25 18 81 74 70 23 71 58 58]
 [19 44 23 81 54 25 81 23 25 81]
 [44 61 23 19 70 23 25 23 72 19]
 [23 81 54 74 23 65 54 31 74 72]
 [62 25 46 23 54 74 70 23 81 74]
 [43 62 62 31 19 44 23 25 44 61]
 [78 44 44 25 23 54 25 61 23 62]
 [49 48 19 44 62 75 59 73 23 80]]


### Defining the network with PyTorch
Model Structure

In [12]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

Time to train

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

Instantiating the model

In [15]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [16]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2618... Val Loss: 3.1960
Epoch: 1/20... Step: 20... Loss: 3.1475... Val Loss: 3.1330
Epoch: 1/20... Step: 30... Loss: 3.1435... Val Loss: 3.1220
Epoch: 1/20... Step: 40... Loss: 3.1156... Val Loss: 3.1184
Epoch: 1/20... Step: 50... Loss: 3.1398... Val Loss: 3.1169
Epoch: 1/20... Step: 60... Loss: 3.1170... Val Loss: 3.1146
Epoch: 1/20... Step: 70... Loss: 3.1052... Val Loss: 3.1120
Epoch: 1/20... Step: 80... Loss: 3.1157... Val Loss: 3.1042
Epoch: 1/20... Step: 90... Loss: 3.1037... Val Loss: 3.0857
Epoch: 1/20... Step: 100... Loss: 3.0630... Val Loss: 3.0490
Epoch: 1/20... Step: 110... Loss: 2.9998... Val Loss: 2.9731
Epoch: 1/20... Step: 120... Loss: 2.8543... Val Loss: 2.8926
Epoch: 1/20... Step: 130... Loss: 2.8270... Val Loss: 2.7789
Epoch: 2/20... Step: 140... Loss: 2.7190... Val Loss: 2.6726
Epoch: 2/20... Step: 150... Loss: 2.6346... Val Loss: 2.5805
Epoch: 2/20... Step: 160... Loss: 2.5563... Val Loss: 2.5089
Epoch: 2/20... Step: 170... Loss:

Checkpoint

In [0]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

Making Predictions

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

Priming and generating text

In [0]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [20]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna was strength on its laughter
and him.

"Well, what have your face, and there's no
house of the collar was there, and
I'm gratiful to me?..."

And seeing Levin and
her life at one of
the croad, the marriage with her eyes. He was said as the commitse face
on his study was asked as she could not trink take him a colver and still things.

"There was in a little
forest to the sole princess, and the same as he saw is a
grist and are, why they was satisfied at
her since that this step as stopping in the chair were talking of in it the principle to a station of his first. All that were, he was so truck at it. But that would be so fact in the feelings in which I should be singered in his hands, and she was so life and saying about it. He has been angry into take them in a sort of privious sense. And he well
and mistaken. The day he
was at the princess, to be
satisfied is now struggling. He came to the cause afder all the children. But
two wife they say that
she's saying to the matter and h

Loading a checkpoint

In [21]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [22]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said to himself.

"What is
it was a longer. I can't be so still fallood, but the same say, happening to
thinking
of the maid. But he'd not a letter and dirtuges at the crup triushes in the conversation.

"Yes, and I went in
were the man, too, to say this with the
pretty. And I will lone to meet your less thoughts of more, to blame
all the mirdly
askants with you he would be simply as
all
that they see it. And I'm not chain of three people of the chair at the carriage of this meetings; but how did you know what I can't say what is not truting them. He has strong out on the man, and
I can't believe her."

Af the man was a cold and to the book.

Alexey Alexandrovitch saw her
short with the carryance. He was
sporting in his book,
the same side of wife and her his brack of the day of the
previous part to
her.

"Yes, there see somewhere? I'd been startfully. That's a picillary of
the same." She wat handing his hands and without the sinner soul to the plict that there are not and al